# Using precalculated samples arrays within the Brightway2 environment

Pascal Lesage, CIRAIG  
March 2018  

Based on an examples published in:  

Lesage, P., C. Mutel, U. Schenker and M. Margni (2018). Uncertainty analysis in LCA using precalculated aggregated datasets. The International Journal of Life Cycle Assessment. https://doi.org/10.1007/s11367-018-1444-x

-----------

## Objectives  
Show how precalculated LCIA arrays generated with the `database_wide_monte_carlo` can be used to calculate LCA scores.  
The example uses `Brightway2` and the `presamples` package.

## Fictional case study - the aluminium water bottle  
 - 150 gram water bottle made of secondary aluminium (new scrap)
 - Cradle-to-gate  
 - Only two unit processes: secondary aluminium, cold-impact extrusion.  
 - Other inputs (such as an interior polymer coating, bottle painting and packaging) are excluded from the simple example.
 - https://www.youtube.com/watch?v=ifl0-re-tV8#t=1m15s  
 - Impact method of interest: IPCC2013 GWP 100a

## Methodology

### Setting up the Brightway project and basic imports

In [1]:
from brightway2 import *

In [2]:
import numpy as np
import pandas as pd
import os
import pyprind
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
iterations = 5

In [ ]:
def delete_and_setup(project_name, ecospold_dirpath, database_name):
    if project_name in projects:
        projects.delete_project(project_name, delete_dir=True)
    projects.set_current(project_name)
    bw2setup()
    importer = SingleOutputEcospold2Importer(
        dirpath=ecospold_dirpath,
        db_name = database_name
    )
    importer.apply_strategies()
    importer.write_database()
    return None

In [ ]:
delete_and_setup('my_mcs_agg', r'C:\mypy\data\ecoinvent_spolds\ecoinvent34\datasets', 'db_UP')

Creating default biosphere



Writing activities to SQLite3 database:
0%                          100%
[####                          ] | ETA: 00:00:00

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applied 2 strategies in 0.01 seconds


[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 03/20/2018 23:09:03
  Finished: 03/20/2018 23:09:05
  Total time elapsed: 00:00:01
  CPU %: 54.50
  Memory %: 0.97
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 4 strategies in 2.04 seconds


In [ ]:
databases

In [ ]:
db_UP = Database('db_UP')

In [ ]:
if 'bottle example database' in databases: 
    Database('bottle example database').delete()
    Database('bottle example database').deregister()

### LCA without the presampled arrays

#### Identify the inputs

In [ ]:
aluminium_UP = act = [act for act in db_UP if 
                   "treatment of aluminium scrap, post-consumer, prepared for recycling, at refiner" in act['name']
                   and 'aluminium, cast alloy' in act['reference product']
                   and act['location']=='RER'
                  ][0]
aluminium_UP

In [ ]:
# Cold impacts extrusion
extrusion_UP = [act for act in db_UP
             if 'impact extrusion of aluminium, 1 stroke' in act['name']
            and act['location']=='RER'][0]
extrusion_UP

#### Defining the new water bottle activity programmatically

In [ ]:
bottle_example_db = Database('bottle example database UP')
bottle_example_db.write(
    {('bottle example database UP', 'aluminium bottle production'): 
        {'name': 'aluminium bottle',
         'unit': 'amount', 
            'exchanges': 
            [
                {'input': aluminium_UP.key,
                 'amount': 0.150, #weight of the water bottle
                 'type': 'technosphere'},
                {'input': extrusion_UP.key,
                 'amount': 0.150, #weight of the water bottle = amount of impact extrusion required
                 'type': 'technosphere'},
                {'input': ('bottle example database UP', 'aluminium bottle production'),
                 'amount': 1,
                 'type': 'production'}
            ]
        },
    })

In [ ]:
bottle_production_UP = get_activity(('bottle example database UP', 'aluminium bottle production'))
bottle_production_UP

#### Identifying the relevant LCIA impact categories

In [ ]:
cc_method = [m for m in methods if 'IPCC 2013' in str(m)
                                and 'GWP' in str(m)
                                and '100' in str(m)][0]
cc_method

In [ ]:
o3_method = [m for m in methods if 'ILCD' in str(m)
                                and 'human health' in str(m)
                                and 'ozone layer depletion' in str(m)
                                and 'no LT' in str(m)][0]
o3_method

#### Carrying out the LCA on bottle production - unit processes

In [ ]:
# Unit process LCA - Bottle
bottle_lca_UP = LCA({bottle_production_UP:1}, method=cc_method)
bottle_lca_UP.lci()
bottle_lca_UP.lcia()
bottle_lca_UP_cc_score = bottle_lca_UP.score
bottle_lca_UP.switch_method(o3_method)
bottle_lca_UP.lcia()
bottle_lca_UP_o3_score = bottle_lca_UP.score
bottle_lca_UP_cc_score, bottle_lca_UP_o3_score

#### Carrying out the Monte Carlo simulation on bottle production for both impact categories

In [ ]:
mc_bottle_cc_UP = MonteCarloLCA({bottle_production_UP:1}, method=cc_method)
scores_bottle_cc_UP = [next(mc_bottle_cc_UP) for _ in pyprind.prog_bar(range(iterations))]
scores_arr_bottle_cc_UP = np.array(scores_bottle_cc_UP)
scores_arr_bottle_cc_UP

In [ ]:
mc_bottle_o3_UP = MonteCarloLCA({bottle_production_UP:1}, method=o3_method)
scores_bottle_o3_UP = [next(mc_bottle_o3_UP) for _ in pyprind.prog_bar(range(iterations))]
scores_arr_bottle_o3_UP = np.array(scores_bottle_o3_UP)
scores_arr_bottle_o3_UP

In [ ]:
# CC
plt.hist(scores_arr_bottle_cc_UP, histtype='step', normed=True)
plt.xlabel("{}/bottle".format(Method(cc_method).metadata['unit']))
plt.ylabel("Frequency")
plt.legend()
plt.title("Monte Carlo results\n{}\nUnit processes".format(cc_method));

In [ ]:
# O3
plt.hist(scores_arr_bottle_o3_UP, histtype='step', normed=True)
plt.xlabel("{}/bottle".format(Method(o3_method).metadata['unit']))
plt.ylabel("Frequency")
plt.legend()
plt.title("Monte Carlo results\n{}\nUnit processes".format(o3_method));

## Aggregated datasets

The LCIA scores that were generated using `database_wide_monte_carlo` are identical to characterized aggregated LCI datasets. Before we see how to use them (Section 5), we see how to include aggregated datasets in Brightway.

### Augmenting the `biosphere3` database

The LCIA score arrays are expressed in the units of the impact categories (kg CO2-Eq, kg CFC-11-Eq). 

To be able to use the score arrays in LCA calculation (system-->inventory-->LCIA scores), we "trick" Brightway into treating these LCIA scores as elementary flows that have a characterization factor of 1. 

To do this, we first add to the database one elementary flow per impact category that represents a "unit impact per unit impact" characterization factor.  

A function that does this is found [here](https://github.com/PascalLesage/bw_utils/blob/master/augment_biosphere3.py).

To run it, simply copy the file to your computer and run it (no package, sorry).  

In [ ]:
os.chdir(r'C:\mypy\code\bw_utils')

In [ ]:
from augment_biosphere3 import addUnitImpactFlowsToBiosphereAndMethods

In [ ]:
addUnitImpactFlowsToBiosphereAndMethods()

We can see the new characterization factor for climate change: 

In [ ]:
[ef for ef in Method(cc_method).load() if Method(cc_method).get_abbreviation() in ef[0]]

Here is a short function that will copy a unit process and convert it to a characterized aggregated LCI dataset:

In [ ]:
def convert_act_to_score_S(act_UP, db_S, list_methods):
    # Make sure activity and methods exist
    assert isinstance(list_methods, list), "Methods should be a list of methods"
    for m in list_methods:
        assert m in methods, "{} is not a recognized method name"
    assert "Activity" in str(type(act_UP))
    # Create a new database for S datasets if one doesn't already exist
    if db_S not in databases:
        Database(db_S).register()
    # Create a copy of the UP activity in the S database
    act_S = act_UP.copy(database=db_S, code=act_UP.key[1])
    # Remove all exchanges except the production exchange
    for exc in act_S.technosphere():
        exc.delete()
    for exc in act_S.biosphere():
        exc.delete()
    # Create an LCA object to calculate LCIA scores
    lca = LCA({act_UP:1})
    lca.lci()
    # Add LCIA scores for each method
    # The elementary flow key is ('biosphere3', method_abbreviation)
    for m in list_methods:
        new_exc = act_S.new_exchange()
        new_exc['input'] = ('biosphere3',
                            Method(m).get_abbreviation()
                           )
        new_exc['output'] = act_S.key
        lca.switch_method(m)
        lca.lcia()
        new_exc['amount'] = lca.score
        new_exc['type'] = 'biosphere'
        new_exc.save()
    # Save the activity
    act_S.save()
    return act_S

First, let's convert the aluminium dataset:

In [ ]:
aluminium_S = convert_act_to_score_S(aluminium_UP, 'bd_S', [cc_method, o3_method])

In [ ]:
aluminium_S

Let's make sure it does not have any technosphere inputs:

In [ ]:
[*aluminium_S.technosphere()]

It should only have two elementary flows: 

In [ ]:
[*aluminium_S.biosphere()]

The score of the LCA for an aggregated activity should be the same as that of a unit process:

In [ ]:
# Unit process LCA
alu_lca_UP = LCA({aluminium_UP:1}, method=cc_method)
alu_lca_UP.lci()
alu_lca_UP.lcia()
alu_lca_UP_cc_score = alu_lca_UP.score
alu_lca_UP.switch_method(o3_method)
alu_lca_UP.lcia()
alu_lca_UP_o3_score = alu_lca_UP.score

# Aggregated process LCA 
alu_lca_S = LCA({aluminium_S:1}, method=cc_method)
alu_lca_S.lci()
alu_lca_S.lcia()
alu_lca_S_cc_score = alu_lca_S.score
alu_lca_S.switch_method(o3_method)
alu_lca_S.lcia()
alu_lca_S_o3_score = alu_lca_S.score

# Comparison
alu_lca_UP_cc_score/alu_lca_S_cc_score, alu_lca_UP_o3_score/alu_lca_S_o3_score

Let's generate an aggregated dataset for extrusion as well:

In [ ]:
extrusion_S = convert_act_to_score_S(extrusion_UP, 'bd_S', [cc_method, o3_method])

And a new bottle activity that uses the aggregated datasets rather than the unit processes:

In [ ]:
bottle_example_db = Database('bottle example database S')
bottle_example_db.write(
    {('bottle example database S', 'aluminium bottle production'): 
        {'name': 'aluminium bottle',
         'unit': 'amount', 
            'exchanges': 
            [
                {'input': aluminium_S.key,
                 'amount': 0.150, #weight of the water bottle
                 'type': 'technosphere'},
                {'input': extrusion_S.key,
                 'amount': 0.150, #weight of the water bottle = amount of impact extrusion required
                 'type': 'technosphere'},
                {'input': ('bottle example database S', 'aluminium bottle production'),
                 'amount': 1,
                 'type': 'production'}
            ]
        },
    })

In [ ]:
bottle_production_S = get_activity(('bottle example database S', 'aluminium bottle production'))
bottle_production_S

Again, the LCA score for the water bottle should be the same using aggregated datasets as when using the unit process database:

In [ ]:
# Aggregated process LCA - Bottle
bottle_lca_S = LCA({bottle_production_S:1}, method=cc_method)
bottle_lca_S.lci()
bottle_lca_S.lcia()
bottle_lca_S_cc_score = bottle_lca_S.score
bottle_lca_S.switch_method(o3_method)
bottle_lca_S.lcia()
bottle_lca_S_o3_score = bottle_lca_S.score

In [ ]:
bottle_lca_S_cc_score/bottle_lca_UP_cc_score, bottle_lca_S_o3_score/bottle_lca_UP_o3_score 

However, uncertainty analysis cannot be done on the bottle with aggregated datasets: the LCIA scores used in the creation of the activity have no uncertainty: 

In [ ]:
mc_bottle_cc_S = MonteCarloLCA({bottle_production_S:1}, method=cc_method)
scores_bottle_cc_S = [next(mc_bottle_cc_S) for _ in pyprind.prog_bar(range(iterations))]
scores_arr_bottle_cc_S = np.array(scores_bottle_cc_S)
scores_arr_bottle_cc_S

In [ ]:
# CC
plt.hist(scores_arr_bottle_cc_S, histtype='step', normed=True)
plt.xlabel("{}/bottle".format(Method(cc_method).metadata['unit']))
plt.ylabel("Frequency")
plt.legend()
plt.title("Monte Carlo results\n{}\nAggregated datasets".format(cc_method));

## Presamples

The precalculated arrays can be integrated in the LCA calculations by using the `presamples` library (see repo [here](https://github.com/PascalLesage/brightway2-presamples/tree/master)). It was codeveloped by Pascal Lesage (CIRAIG) and Chris Mutel (PSI, main developer of the Brightway2 Framework) as a means to integrate precalculated samples directly in LCA. It has many other applications, but the core motivation at the start was to be able to incude the results of arrays such as those generated by `database_wide_monte_carlo`.

As of March 20, 2018, the `presamples` package can be installed directly using pip. In the conda environment where `Brightway2` is installed, simply type `pip install presamples`. 

To use the precalculated presamples in LCA, follow the following steps:

### Generate the matrix_data

Precalculated samples for use in LCA matrices need to be organized in a list of `(samples, indices, matrix label)`.

* `samples` are a two-dimensional Numpy array, where each row contains values for a specific matrix element that will be replaced and each column contains values for a given realization of the LCA model.
* `indices` is an iterable with row and (usually) column indices. The *ith* element of `indices` refers to the *ith* row of the `samples`. The exact format of indices depends on the matrix to be constructed. These indices tell us where exactly to insert the samples into a specific matrix.
*  `matrix label` is a string giving the name of the matrix to be modified in the LCA class. This string is normally, but not necessarily, one of `('technosphere', 'biosphere', 'cf')`.

In the case of our aggregated LCIA scores, we have four matrix elements for which we want use precalculated:  
* Climate change scores for the aluminium aggregated activity    
* Climate change scores for the extrusion aggregated activity    
* Ozone layer depletion scores for the aluminium aggregated activity    
* Ozone layer depletion scores for the extrusion aggregated activity    

#### Samples  
The samples are basically the four calculated LCIA arrays horizontally stacked, leading to an array of shape (4, 5000).  
Here is how to get them:

In [ ]:
# Directory with LCIA scores
lcia_fp = r'c:/mypy/data/my_mcs/db/results/LCIA'

# The subdirectory name is the abbreviation of the LCIA method
# The file name is the code of the activity (i.e. the second element of the activity's key)
# To generate a single array, as described above:
samples = np.zeros((4, 5000))
for act in [aluminium_S, extrusion_S]:
    for m in [cc_method, o3_method]:
        samples[0, :] = np.load(os.path.join(lcia_fp, Method(m).get_abbreviation(), act.key[1]+'.npy'))
samples.shape